# Load dataset, add the company name, student matriculation number and rename the power consumption column. 

In [ ]:
import pandas as pd

df=pd.read_csv("Documents/datasets/hourlyenergyconsumption/DominionVirginiaPower.csv")
df.insert(0,"Company","Dominion Virginia Power");
df["MatricNo"] = "A0206682A";
df=df.rename(columns={"DOM_MW":"Total_MW"});
df=df.sort_values(df.columns[1])

# Set up the connection to AWS and send data to AWS.

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
import random, time, json

# A random programmatic shadow client ID.
SHADOW_CLIENT = "myShadowClient"

# The unique hostname that &IoT; generated for 
# this device.
HOST_NAME = "ac3gpzrmcfb87-ats.iot.ap-southeast-1.amazonaws.com"

# The relative path to the correct root CA file for &IoT;, 
# which you have already saved onto this device.
ROOT_CA = "/Users/lydoan/Documents/NUS/EE5111/IOTPROJ/AmazonRootCA1.pem"

# The relative path to your private key file that 
# &IoT; generated for this device, which you 
# have already saved onto this device.
PRIVATE_KEY = "/Users/lydoan/Documents/NUS/EE5111/IOTPROJ/274d5b6b8f-private.pem.key"

# The relative path to your certificate file that 
# &IoT; generated for this device, which you 
# have already saved onto this device.
CERT_FILE = "/Users/lydoan/Documents/NUS/EE5111/IOTPROJ/274d5b6b8f-certificate.pem.crt"

# A programmatic shadow handler name prefix.
SHADOW_HANDLER = "A0206682A1"

# Automatically called whenever the shadow is updated.
def myShadowUpdateCallback(payload, responseStatus, token):
  print()
  print('UPDATE: $aws/things/' + SHADOW_HANDLER + 
    '/shadow/update/#')
  print("payload = " + payload)
  print("responseStatus = " + responseStatus)
  print("token = " + token)

## Create, configure, and connect a shadow client.
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY,
  CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(
  SHADOW_HANDLER, True)

# To stop running this script, press Ctrl+C.
while True:
#Update the shadow
    for i in range (6000):
        temp = df.iloc[i]
        tmp = temp.to_dict()
        jsonPayload= {"state":{"reported":tmp}}
        jsonPayload = json.dumps(jsonPayload)
        print (jsonPayload)
        print (i)
        myDeviceShadow.shadowUpdate(jsonPayload,myShadowUpdateCallback, 5)
  # Wait for this test value to be added.
        time.sleep(1)           